### DSC550 Exercise 9.2
### Madison Christiansen

In [71]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

loandf = pd.read_csv('/Users/madisonchristiansen/Desktop/Loan_Train.csv')   
loandf.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [21]:
loandf = loandf.drop(columns=['Loan_ID']) # drop id columns

In [22]:
loandf = loandf.dropna(axis=0) # drop rows with missing values

In [23]:
# dummy vairables for the target column
loandf['Loan_Status_target'] = loandf['Loan_Status'].replace(to_replace=["Y","N"], value=[1,0]) # (Y=1 and N=0)

In [24]:
catagcol = loandf.select_dtypes("object").columns # category columns
catagcol= list(set(catCols))
clean_loandf = pd.get_dummies(loandf, columns=catagcol) # dummy variables 

In [25]:
clean_loandf.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status_target,Gender_Female,Gender_Male,Property_Area_Rural,Property_Area_Semiurban,...,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Married_No,Married_Yes,Loan_Status_N,Loan_Status_Y
1,4583,1508.0,128.0,360.0,1.0,0,0,1,1,0,...,0,1,0,0,1,0,0,1,1,0
2,3000,0.0,66.0,360.0,1.0,1,0,1,0,0,...,1,0,0,0,1,0,0,1,0,1
3,2583,2358.0,120.0,360.0,1.0,1,0,1,0,0,...,1,0,0,0,0,1,0,1,0,1
4,6000,0.0,141.0,360.0,1.0,1,0,1,0,0,...,1,0,0,0,1,0,1,0,0,1
5,5417,4196.0,267.0,360.0,1.0,1,0,1,0,0,...,0,0,1,0,1,0,0,1,0,1


In [48]:
## split train and test set, loan_status_target
X = clean_loandf.drop(['Loan_Status_target', 'Loan_Status_N', 'Loan_Status_Y'], axis = 1)
y = clean_loandf['Loan_Status_target']

X_train, X_test, y_train, y_test = train_test_split(X, y)


In [49]:
## create scale, knn, and pipline
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,1))
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
pipe= Pipeline([("scaler", minmax_scale), ("knn", knn)])

In [50]:
## fit knn to the data with the pipline
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('knn', KNeighborsClassifier(n_jobs=-1))])

In [51]:
y_pred=pipe.predict(X_test) # predict 
pipe.score(X_test, y_test) # accuracy

/Users/madisonchristiansen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/madisonchristiansen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the

0.7333333333333333

In [73]:
search_space = [{"knn__n_neighbors": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}]

In [74]:
# standardizer 
standardizer = StandardScaler()
features_standardized = standardizer.fit_transform(X)

In [75]:
# grid search 
classifier = GridSearchCV(pipe, search_space, cv=5, verbose=0).fit(features_standardized, y)
classifier.best_estimator_.get_params()["knn__n_neighbors"]

/Users/madisonchristiansen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/madisonchristiansen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the

/Users/madisonchristiansen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/madisonchristiansen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the

/Users/madisonchristiansen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
/Users/madisonchristiansen/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the

3

In [76]:
print("hyperparameter values: {}".format(classifier.best_params_))

## accuracy of the grid search best model on the test set
print("accuracy: {:.2f}%".format(classifier.best_score_*100))

hyperparameter values: {'knn__n_neighbors': 3}
accuracy: 72.29%


In [97]:

pipe = Pipeline([('standardizer', StandardScaler()), ("classifier", KNeighborsClassifier(n_neighbors=5, n_jobs=-1))])

# 12.3 hyperparameter , expand your search space to include logistic regression and random forest models 
search_space2 = [{"classifier": [LogisticRegression()],
"classifier__penalty": ['l1', 'l2'],
"classifier__C": np.logspace(0, 4, 10)},
{"classifier": [RandomForestClassifier()],
"classifier__n_estimators": [10, 100, 1000],
"classifier__max_features": [1, 2, 3]}]

# Create grid search
gridsearch = GridSearchCV(pipe, search_space2, cv=5, verbose=0)
# Fit grid search
bestmodel = gridsearch.fit(X_train, y_train)


/Users/madisonchristiansen/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 145.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/madisonchristiansen/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/madisonchristiansen/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/Users/madisonchristiansen/opt/anaconda3/lib/python3.9/site-packages/skl

In [98]:
bestmodel.best_estimator_.get_params()

{'memory': None,
 'steps': [('standardizer', StandardScaler()),
  ('classifier', LogisticRegression())],
 'verbose': False,
 'standardizer': StandardScaler(),
 'classifier': LogisticRegression(),
 'standardizer__copy': True,
 'standardizer__with_mean': True,
 'standardizer__with_std': True,
 'classifier__C': 1.0,
 'classifier__class_weight': None,
 'classifier__dual': False,
 'classifier__fit_intercept': True,
 'classifier__intercept_scaling': 1,
 'classifier__l1_ratio': None,
 'classifier__max_iter': 100,
 'classifier__multi_class': 'auto',
 'classifier__n_jobs': None,
 'classifier__penalty': 'l2',
 'classifier__random_state': None,
 'classifier__solver': 'lbfgs',
 'classifier__tol': 0.0001,
 'classifier__verbose': 0,
 'classifier__warm_start': False}

In [1]:
print("Best Model accuracy: {:.2f}%".format(bestmodel.best_score_*100))

NameError: name 'bestmodel' is not defined

### The Logistic Regression is the best modle to use for this dataset with the accuracy being 80%